In [1]:
import torch
from modules import SConv2d, SLinear, SModel, SReLU
from models import SCrossEntropyLoss

In [26]:
fc1 = SLinear(3,3,bias=False)
fc1.op.weight.data = torch.Tensor(list(range(1,10))).view(fc1.op.weight.shape) * 0.1
# fc1.op.bias.data = torch.zeros_like(fc1.op.bias)
fc2 = SLinear(3,3,bias=False)
fc2.op.weight.data = torch.Tensor(list(range(9,0,-1))).view(fc2.op.weight.shape) * 0.1
fc2.op.weight.data[0,0] = 2
# fc2.op.bias.data = torch.zeros_like(fc2.op.bias)
relu = SReLU()
criteria = SCrossEntropyLoss()
optimizer = torch.optim.SGD(list(fc1.parameters()) + list(fc2.parameters()), 1e-1)

# for _ in range(1000):
#     optimizer.zero_grad()
#     x = torch.Tensor([[1,2,3]])
#     xS = torch.zeros_like(x)
#     labels = torch.LongTensor([0])

#     x1, xS1 = fc1(x, xS)
#     x1.retain_grad()
#     xS1.retain_grad()
#     x2, xS2 = relu(x1, xS1)
#     x2.retain_grad()
#     xS2.retain_grad()
#     x3, xS3 = fc2(x2, xS2)
#     x3.retain_grad()
#     xS3.retain_grad()

#     loss = criteria(x3, xS3, labels)
#     loss.backward()
#     optimizer.step()

optimizer = torch.optim.SGD(list(fc1.parameters()) + list(fc2.parameters()), 0)
optimizer.zero_grad()
x = torch.Tensor([[1,2,3]])
# x = torch.Tensor([[1,2,3], [1,2,3]])
xS = torch.zeros_like(x)
labels = torch.LongTensor([0])
# labels = torch.LongTensor([0,0])

x1, xS1 = fc1(x, xS)
x1.retain_grad()
xS1.retain_grad()
x2, xS2 = relu(x1, xS1)
x2.retain_grad()
xS2.retain_grad()
x3, xS3 = fc2(x2, xS2)
x3.retain_grad()
xS3.retain_grad()

loss = criteria(x3, xS3, labels)
loss.backward()
optimizer.step()
print(loss)

tensor(0.0126, grad_fn=<SCrossEntropyLossFunctionBackward>)


In [27]:
# x3SG = torch.Tensor([[0.0123928,-0.0117341,-0.000658689,-0.0117341,0.011742,-7.92681E-6,-0.000658689,-7.92681E-6,0.000666616]]).view(1,3,3)
# x1SG = torch.Tensor([[0.0249031,0.00560088,0.00560088,0.00560088,0.00129391,0.00129391,0.00560088,0.00129391,0.00129391]]).view(1,3,3)

In [28]:
x3

tensor([[8.8600, 4.4400, 1.5600]], grad_fn=<SLinearFunctionBackward>)

In [32]:
def CECross(output):
    BS = output.shape[0]
    e3 = (output - output.max()).exp()
    e3_sum = e3.sum(dim=1)
    ratio = (e3 / e3_sum).view(BS,1,-1)
    return (torch.diag_embed(ratio.view(BS,-1),0,1) - ratio.swapaxes(1,2).bmm(ratio))
x3SG = CECross(x3)

In [33]:
x3SG

tensor([[[ 1.2393e-02, -1.1734e-02, -6.5869e-04],
         [-1.1734e-02,  1.1742e-02, -7.9268e-06],
         [-6.5869e-04, -7.9268e-06,  6.6662e-04]]], grad_fn=<SubBackward0>)

In [35]:
x1SG = fc2.op.weight.t().matmul(x3SG).matmul(fc2.op.weight)

In [36]:
x1SG

tensor([[[0.0249, 0.0056, 0.0056],
         [0.0056, 0.0013, 0.0013],
         [0.0056, 0.0013, 0.0013]]], grad_fn=<UnsafeViewBackward>)

In [37]:
def crossSecond(IN, outS):
    BS = IN.shape[0]
    IN = IN.view(BS,1,-1)
    return IN.swapaxes(1,2).bmm(IN).view(BS,1,-1).swapaxes(1,2).bmm(outS.view(BS,1,-1))
fc1WSG = crossSecond(x,  x1SG)
fc2WSG = crossSecond(x2, x3SG)

In [38]:
fc1WSG

tensor([[[0.0249, 0.0056, 0.0056, 0.0056, 0.0013, 0.0013, 0.0056, 0.0013,
          0.0013],
         [0.0498, 0.0112, 0.0112, 0.0112, 0.0026, 0.0026, 0.0112, 0.0026,
          0.0026],
         [0.0747, 0.0168, 0.0168, 0.0168, 0.0039, 0.0039, 0.0168, 0.0039,
          0.0039],
         [0.0498, 0.0112, 0.0112, 0.0112, 0.0026, 0.0026, 0.0112, 0.0026,
          0.0026],
         [0.0996, 0.0224, 0.0224, 0.0224, 0.0052, 0.0052, 0.0224, 0.0052,
          0.0052],
         [0.1494, 0.0336, 0.0336, 0.0336, 0.0078, 0.0078, 0.0336, 0.0078,
          0.0078],
         [0.0747, 0.0168, 0.0168, 0.0168, 0.0039, 0.0039, 0.0168, 0.0039,
          0.0039],
         [0.1494, 0.0336, 0.0336, 0.0336, 0.0078, 0.0078, 0.0336, 0.0078,
          0.0078],
         [0.2241, 0.0504, 0.0504, 0.0504, 0.0116, 0.0116, 0.0504, 0.0116,
          0.0116]]], grad_fn=<BmmBackward0>)

In [39]:
fc2WSG

tensor([[[ 2.4290e-02, -2.2999e-02, -1.2910e-03, -2.2999e-02,  2.3014e-02,
          -1.5537e-05, -1.2910e-03, -1.5537e-05,  1.3066e-03],
         [ 5.5519e-02, -5.2569e-02, -2.9509e-03, -5.2569e-02,  5.2604e-02,
          -3.5512e-05, -2.9509e-03, -3.5512e-05,  2.9864e-03],
         [ 8.6749e-02, -8.2138e-02, -4.6108e-03, -8.2138e-02,  8.2194e-02,
          -5.5488e-05, -4.6108e-03, -5.5488e-05,  4.6663e-03],
         [ 5.5519e-02, -5.2569e-02, -2.9509e-03, -5.2569e-02,  5.2604e-02,
          -3.5512e-05, -2.9509e-03, -3.5512e-05,  2.9864e-03],
         [ 1.2690e-01, -1.2016e-01, -6.7450e-03, -1.2016e-01,  1.2024e-01,
          -8.1170e-05, -6.7450e-03, -8.1170e-05,  6.8261e-03],
         [ 1.9828e-01, -1.8774e-01, -1.0539e-02, -1.8774e-01,  1.8787e-01,
          -1.2683e-04, -1.0539e-02, -1.2683e-04,  1.0666e-02],
         [ 8.6749e-02, -8.2138e-02, -4.6108e-03, -8.2138e-02,  8.2194e-02,
          -5.5488e-05, -4.6108e-03, -5.5488e-05,  4.6663e-03],
         [ 1.9828e-01, -1.8774e-01

In [10]:
fc1WSG.view(9,9).diag().view(-1).sort()[1]

tensor([4, 5, 7, 1, 3, 8, 6, 2, 0])

In [11]:
fc1.weightS.grad.view(-1).sort()[1]

tensor([6, 3, 7, 4, 0, 8, 5, 1, 2])

In [64]:
O = torch.ones(3,2,2)
I = torch.ones(3,10)
a = torch.nn.Linear(10,2)
a.weight.t().matmul(O).matmul(a.weight).shape

torch.Size([3, 10, 10])

In [65]:
I.mm(a.weight.t())

tensor([[-0.7404, -0.0510],
        [-0.7404, -0.0510],
        [-0.7404, -0.0510]], grad_fn=<MmBackward>)